实验要求:
1. 数据分析、统计
   1. 单个特征的分布
   2. 统计缺失值
   3. 特征间的相关性
   4. 推测特征的含义
   5. 异常样本
   6. 数据抽样
2. 特征抽取
   1. 特征的变换,如str转int,取log
   2. 尝试组合特征
   3. 特征子集选择


In [247]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
%matplotlib inline

In [138]:
raw_path = "data/pica2015.csv"
cleaned_path = "data/pica2015_cleaned.csv"

In [139]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

Below is wash data process

1. use mode data to fill the NaN
2. turn `str` and `object` to `int` 
3. drop columns with same data of all the student, thus unrelated to the classification result, e.g. , columns[5:19]

After the first run, the clean data has been loaded into "data/pica2015_cleaned.csv"

So below can be skipped until the `finish` sign

In [140]:
df = pd.read_csv(raw_path)

C:\Users\x\AppData\Local\Temp\ipykernel_36796\1462437030.py:1: DtypeWarning: Columns (86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,244) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_path)


In [141]:
# actually, we can see from the codebook that columns[5:19] are unrelated to the target variable
# so we can drop them
df.drop(columns=df.columns[5:19], inplace=True)

In [142]:
# temporarily turn nan to -1 for type conversion
df.replace(' ', -1, inplace=True)

In [143]:
# turn column[3] to str
df.iloc[:, 3] = df.iloc[:, 3].astype(str)

In [144]:
# turn the q part to `int`
df.iloc[:, 7:231] = df.iloc[:, 7:231].astype(int)

In [145]:
# turn the v part to `float`
df.iloc[:, 231:-1] = df.iloc[:, 231:-1].astype(float)

In [146]:
# see the features dtypes
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32130 entries, 0 to 32129
Data columns (total 416 columns):
 #    Column       Dtype  
---   ------       -----  
 0    No           int64  
 1    CNTSCHID     int64  
 2    Region       int64  
 3    STRATUM      object 
 4    SUBNATIO     int64  
 5    CBASCI       int64  
 6    BOOKID       int64  
 7    ST001D01T    int32  
 8    ST003D02T    int32  
 9    ST003D03T    int32  
 10   ST004D01T    int32  
 11   ST005Q01TA   int32  
 12   ST006Q01TA   int32  
 13   ST006Q02TA   int32  
 14   ST006Q03TA   int32  
 15   ST006Q04TA   int32  
 16   ST007Q01TA   int32  
 17   ST008Q01TA   int32  
 18   ST008Q02TA   int32  
 19   ST008Q03TA   int32  
 20   ST008Q04TA   int32  
 21   ST011Q01TA   int32  
 22   ST011Q02TA   int32  
 23   ST011Q03TA   int32  
 24   ST011Q04TA   int32  
 25   ST011Q05TA   int32  
 26   ST011Q06TA   int32  
 27   ST011Q07TA   int32  
 28   ST011Q08TA   int32  
 29   ST011Q09TA   int32  
 30   ST011Q10TA   int32  

In [147]:
# actually null values are ' ' (space), need to change to NaN
df.replace(-1, np.nan, inplace=True)

In [148]:
# see null values count
df.isnull().sum()

No               0
CNTSCHID         0
Region           0
STRATUM          0
SUBNATIO         0
CBASCI           0
BOOKID           0
ST001D01T        0
ST003D02T        0
ST003D03T        0
ST004D01T        0
ST005Q01TA       0
ST006Q01TA       0
ST006Q02TA       0
ST006Q03TA       0
ST006Q04TA       0
ST007Q01TA       0
ST008Q01TA       0
ST008Q02TA       0
ST008Q03TA       0
ST008Q04TA       0
ST011Q01TA       0
ST011Q02TA       0
ST011Q03TA       0
ST011Q04TA       0
ST011Q05TA       0
ST011Q06TA       0
ST011Q07TA       0
ST011Q08TA       0
ST011Q09TA       0
ST011Q10TA       0
ST011Q11TA       0
ST011Q12TA       0
ST011Q16NA       0
ST011D17TA     163
ST011D18TA     282
ST011D19TA     379
ST012Q01TA       0
ST012Q02TA       0
ST012Q03TA       0
ST012Q05NA       0
ST012Q06NA       0
ST012Q07NA       0
ST012Q08NA       0
ST012Q09NA       0
ST013Q01TA       0
ST123Q01NA       0
ST123Q02NA       0
ST123Q03NA       0
ST123Q04NA       0
ST019AQ01T       0
ST019BQ01T       0
ST019CQ01T  

In [149]:
# search all the column name with null values
null_columns = df.columns[df.isnull().any()]
null_columns

Index(['ST011D17TA', 'ST011D18TA', 'ST011D19TA', 'ST121Q01NA', 'ST121Q02NA',
       'ST121Q03NA', 'ST082Q01NA', 'ST082Q02NA', 'ST082Q03NA', 'ST082Q08NA',
       ...
       'ST078Q03NA', 'ST078Q04NA', 'ST078Q05NA', 'ST078Q06NA', 'ST078Q07NA',
       'ST078Q08NA', 'ST078Q09NA', 'ST078Q10NA', 'ST078Q11NA', 'ST065Class'],
      dtype='object', length=162)

In [150]:
# fill the null values with the mode of the column
mode_dict = df[null_columns].mode().iloc[0].astype(int).to_dict()

In [151]:
df.fillna(mode_dict, inplace=True)

In [152]:
# check all the nan have been filled
df.isnull().sum()

No             0
CNTSCHID       0
Region         0
STRATUM        0
SUBNATIO       0
CBASCI         0
BOOKID         0
ST001D01T      0
ST003D02T      0
ST003D03T      0
ST004D01T      0
ST005Q01TA     0
ST006Q01TA     0
ST006Q02TA     0
ST006Q03TA     0
ST006Q04TA     0
ST007Q01TA     0
ST008Q01TA     0
ST008Q02TA     0
ST008Q03TA     0
ST008Q04TA     0
ST011Q01TA     0
ST011Q02TA     0
ST011Q03TA     0
ST011Q04TA     0
ST011Q05TA     0
ST011Q06TA     0
ST011Q07TA     0
ST011Q08TA     0
ST011Q09TA     0
ST011Q10TA     0
ST011Q11TA     0
ST011Q12TA     0
ST011Q16NA     0
ST011D17TA     0
ST011D18TA     0
ST011D19TA     0
ST012Q01TA     0
ST012Q02TA     0
ST012Q03TA     0
ST012Q05NA     0
ST012Q06NA     0
ST012Q07NA     0
ST012Q08NA     0
ST012Q09NA     0
ST013Q01TA     0
ST123Q01NA     0
ST123Q02NA     0
ST123Q03NA     0
ST123Q04NA     0
ST019AQ01T     0
ST019BQ01T     0
ST019CQ01T     0
ST021Q01TA     0
ST022Q01TA     0
ST124Q01TA     0
ST125Q01NA     0
ST126Q01TA     0
ST127Q01TA    

In [252]:
df.shape

(32130, 416)

Below is to see the corresponding meaning of each columns and save to dicts

In [186]:
cb = pd.read_excel('data/codebook.xlsx')
cb.NAME = cb.NAME.astype(str)

In [201]:
cb.drop(index = cb[cb.NAME=='nan'].index, inplace=True)

In [202]:
cb

,NAME,VARLABEL,TYPE,FORMAT,VARNUM,MINMAX,VAL,LABEL,COUNT,PERCENT
0,CNTRYID,Country Identifier,NUM,3.0,1.0,8 - 974,NaN,NaN,NaN,NaN
75,CNT,Country code 3-character,CHAR,$3.,2.0,NaN,NaN,NaN,NaN,NaN
150,CNTSCHID,Intl. School ID,NUM,8.0,3.0,800001 - 97400238,NaN,NaN,NaN,NaN
152,CNTSTUID,Intl. Student ID,NUM,8.0,4.0,800001 - 97407472,NaN,NaN,NaN,NaN
154,CYC,PISA Assessment Cycle (2 digits + 2 character ...,CHAR,$4.,5.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11221,PV8SSES,Plausible Value 8 in System Subscale of Scienc...,NUM,8.3,917.0,34.746 - 935.05,NaN,NaN,NaN,NaN
11223,PV9SSES,Plausible Value 9 in System Subscale of Scienc...,NUM,8.3,918.0,12.619 - 928.349,NaN,NaN,NaN,NaN
11225,PV10SSES,Plausible Value 10 in System Subscale of Scien...,NUM,8.3,919.0,16.907 - 898.335,NaN,NaN,NaN,NaN
11227,SENWT,Senate Weight (sum of 5000 per country),NUM,12.5,920.0,0.00375 - 12.89587,NaN,NaN,NaN,NaN


In [213]:
col = df.columns.to_numpy()
name = cb.NAME.to_numpy()

In [224]:
zai = []
for i in range(len(name)):
    if name[i] in col:
        zai.append(False)
    else:
        zai.append(True)
zai

[True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fa

In [225]:
cb.drop(index = cb[zai].index, inplace=True)

In [231]:
# load cb's "NAME" and "VARLABEL" to a dict
cb_dict = cb.set_index('NAME').VARLABEL.to_dict()

In [250]:
# load cb_dict to "data/dictionary_cleaned.txt"
# need to add "NO" column by hand
cb_dict_path = "data/dictionary_cleaned.txt"
f = open(cb_dict_path, 'w')
f.write(json.dumps(cb_dict, indent=0))

36173

save dicts `finished`

In [261]:
df.iloc[:, 96:101]

,ST059Q02TA,ST059Q03TA,ST060Q01NA,ST061Q01NA,ST062Q01TA
0,2.0,4.0,56.0,75.0,2.0
1,0.0,0.0,99.0,60.0,1.0
2,4.0,3.0,27.0,50.0,1.0
3,3.0,2.0,30.0,60.0,2.0
4,3.0,4.0,20.0,60.0,2.0
...,...,...,...,...,...
32125,2.0,2.0,30.0,45.0,1.0
32126,5.0,4.0,40.0,55.0,1.0
32127,4.0,3.0,14.0,55.0,1.0
32128,5.0,4.0,16.0,60.0,1.0


In [254]:
# now we can use cb_dict to drop some definitely unrelated columns by hand
df.drop(index=[], inplace=True)

Clean `finished` 

Below are done on cleaned data. 

In [ ]:
df = pd.read_csv(cleaned_path)

In [ ]:
# divide into 3 groups: p(personal), q(question), v(value)
p = df.iloc[:,0:7]
q = df.iloc[:,7:231]
v = df.iloc[:,231:-1]

In [ ]:
df.shape

In [ ]:
# see some noise values, like 'not apllicable', 'invalid','no response', 'missing', etc.
v_c = []
for i in range(7, 415):
    v_c.append(df.iloc[:,i].value_counts())
print(v_c)